In [1]:
# General
import os
from random import randint

# Weights & Biases
import wandb
from pytorch_lightning.loggers import WandbLogger

# Pytorch modules
import torch
from torch.nn import functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split, Dataset

# Pytorch-Lightning
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics # a new pakage for torchmetrics
from torchmetrics.functional import accuracy

# sci-kit learn and scikit-image
import sklearn
import skimage

# Dataset
from torchvision.datasets import MNIST ######### not required
from torchvision import transforms

# number of CPUs
# cpu_count = 0 if torch.cuda.is_available() else os.cpu_count()

# use GPU tensor cores
torch.set_float32_matmul_precision('high')

# Load data

In [2]:
class MNISTDataModule(LightningDataModule):

    def __init__(self, data_dir='./', batch_size=256):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.ToTensor()

    def prepare_data(self):
        '''called only once and on 1 GPU'''
        # download data
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        '''called on each GPU separately - stage defines if we are at fit or test step'''
        # we set up only relevant datasets when stage is specified (automatically set by Pytorch-Lightning)
        if stage == 'fit' or stage is None:
            mnist_train = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        '''returns training dataloader'''
        mnist_train = DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle = True)#, num_workers = cpu_count)
        return mnist_train

    def val_dataloader(self):
        '''returns validation dataloader'''
        mnist_val = DataLoader(self.mnist_val, batch_size=self.batch_size, shuffle = True)#, num_workers = cpu_count)
        return mnist_val

    def test_dataloader(self):
        '''returns test dataloader'''
        mnist_test = DataLoader(self.mnist_test, batch_size=self.batch_size, shuffle = True)#, num_workers = cpu_count)
        return mnist_test

In [3]:
# declaring the path of the train and test folders
train_path = "DATASET/TRAIN"
test_path = "DATASET/TEST"
classes_dir_data = os.listdir(train_path)
num_of_classes = len(classes_dir_data)
print("Total Number of Classes :" , num_of_classes)
num = 0
classes_dict = {}
classes_lst = []
num_dict = {}
for c in  classes_dir_data:
    classes_dict[c] = num
    num_dict[num] = c
    classes_lst.append(c)
    num = num +1
"""
num_dict contains a dictionary of the classes numerically and it's corresponding classes.
classes_dict contains a dictionary of the classes and the coresponding values numerically.
"""
num_of_classes = len(classes_dir_data)

classes_dict

Total Number of Classes : 10


{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [4]:
#creating the dataset

#dataset

class Image_Dataset(Dataset):

    def __init__(self,classes,image_base_dir,transform = None, target_transform = None):

        """

        classes:The classes in the dataset

        image_base_dir:The directory of the folders containing the images

        transform:The trasformations for the Images

        Target_transform:The trasformations for the target

        """

        self.img_labels = classes

        self.imge_base_dir = image_base_dir

        self.transform = transform

        self.target_transform = target_transform

    def __len__(self):

        return len(self.img_labels)

    def __getitem__(self,idx):

        img_dir_list = os.listdir(os.path.join(self.imge_base_dir,self.img_labels[idx]))

        image_path = img_dir_list[randint(0,len(img_dir_list)-1)]

        #print(image_path)

        image_path = os.path.join(self.imge_base_dir,self.img_labels[idx],image_path)

        image = skimage.io.imread(image_path)

        if self.transform:

            image = self.transform(image)

        if self.transform:

            label = self.target_transform(self.img_labels[idx])

        return image,label

In [5]:
size = 50 # need to be the same as what is used in layer_5/ input layer ot the cnn

basic_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
        transforms.Grayscale(1),
    transforms.ToTensor()])
training_transformations = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((size,size)),
    transforms.RandomRotation(degrees = 45),
    transforms.RandomHorizontalFlip(p = 0.005),
        transforms.Grayscale(1),
    transforms.ToTensor()
])

def target_transformations(x):
    return torch.tensor(classes_dict.get(x))

In [6]:
class YogaDataModule(pl.LightningDataModule):

    def __init__(self):
            super().__init__()            

    def prepare_data(self):
        self.train = Image_Dataset(classes_dir_data,train_path,training_transformations,target_transformations)
        self.valid = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)
        self.test = Image_Dataset(classes_dir_data,test_path,basic_transformations,target_transformations)

    def train_dataloader(self):
        return DataLoader(self.train,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def val_dataloader(self):  
        return DataLoader(self.valid,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

    def test_dataloader(self):
        return DataLoader(self.test,batch_size = 64,shuffle = True)#False, num_workers = cpu_count)

# Define Model

In [7]:
class LitMNIST(LightningModule):

    def __init__(self, n_classes=10, acc_task="multiclass", n_layer_1=128, n_layer_2=256, lr=1e-3):
        '''method used to define our model parameters'''
        super().__init__()
        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, n_layer_1)
        self.layer_2 = torch.nn.Linear(n_layer_1, n_layer_2)
        self.layer_3 = torch.nn.Linear(n_layer_2, n_classes)
        # optimizer parameters
        self.lr = lr
        # metrics
        self.acc_task = acc_task
        self.n_classes = n_classes
        self.accuracy = torchmetrics.Accuracy(task=self.acc_task, num_classes=self.n_classes)
        self.class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self, x):
        '''method used for inference input -> output'''
        batch_size, channels, width, height = x.size()
        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        '''needs to return a loss from a single batch'''
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # Log training loss
        self.log('train_loss', loss)
        # Log metrics
        self.log('train_acc', self.accuracy(logits, y))
        return loss

    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # Log validation loss (will be automatically averaged over an epoch)
        self.log('valid_loss', loss)
        # Log metrics
        self.log('valid_acc', self.accuracy(logits, y))
        self.cpu_logits = logits.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"valid_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_logits,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})

    def test_step(self, batch, batch_idx):
        '''used for logging metrics'''
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # Log test loss
        self.log('test_loss', loss)
        # Log metrics
        self.log('test_acc', self.accuracy(logits, y))
        self.cpu_logits = logits.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"test_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_logits,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
    
    def configure_optimizers(self):
        '''defines model optimizer'''
        return Adam(self.parameters(), lr=self.lr)

In [8]:
class YogaModel(LightningModule):

    def __init__(self, n_classes=10, acc_task="multiclass", lr=1e-3):
        super().__init__()
        """
        The convolutions are arranged in such a way that the image maintain the x and y dimensions. only the channels change
        """
        self.layer_1 = nn.Conv2d(in_channels = 1,out_channels = 3,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_2 = nn.Conv2d(in_channels = 3,out_channels = 6,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_3 = nn.Conv2d(in_channels = 6,out_channels = 12,kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (3,3),padding = (1,1),stride = (1,1))
        self.layer_5 = nn.Linear(12*50*50,1000)#the input dimensions are (Number of dimensions * height * width)
        self.layer_6 = nn.Linear(1000,100)
        self.layer_7 = nn.Linear(100,50)
        self.layer_8 = nn.Linear(50,10)
        self.layer_9 = nn.Linear(10,5)
        self.lr = lr
        # metrics
        self.acc_task = acc_task
        self.n_classes = n_classes
        self.accuracy = torchmetrics.Accuracy(task=self.acc_task, num_classes=self.n_classes)
        self.class_names = classes_lst
        # optional - save hyper-parameters to self.hparams
        # they will also be automatically logged as config parameters in W&B
        self.save_hyperparameters()

    def forward(self,x):
        """
        x is the input data
        """
        x = self.layer_1(x)
        x = self.pool(x)
        x = self.layer_2(x)
        x = self.pool(x)
        x = self.layer_3(x)
        x = self.pool(x)
        x = x.view(x.size(0),-1)
        print(x.size())
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        x = self.layer_9(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = self.lr)
        return optimizer

# The Pytorch-Lightning module handles all the iterations of the epoch

    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('train_loss', loss)
        # Log metrics
        self.log('train_acc', self.accuracy(y_pred, y))
        return loss

    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('val_loss', loss)
        # Log metrics
        self.log('val_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"val_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return loss

    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # Log training loss
        self.log('test_loss', loss)
        # Log metrics
        self.log('test_acc', self.accuracy(y_pred, y))
        self.cpu_pred = y_pred.to("cpu").detach().numpy()
        self.cpu_y = y.to("cpu").detach().numpy()
        wandb.log({"test_conf_mat" : wandb.plot.confusion_matrix(probs=self.cpu_pred,
                        y_true=self.cpu_y, preds=None,
                        class_names=self.class_names)})
        return loss

# Single training run

In [9]:
wandb.login()
wandb_logger = WandbLogger(project='MNIST')

# TRAIN
# setup data
# data = MNISTDataModule()
data = YogaDataModule()

# setup model - choose different hyperparameters per experiment
# model = LitMNIST(n_layer_1=128, n_layer_2=256, lr=1e-3, n_classes=10)
model = YogaModel(n_classes=num_of_classes)

trainer = Trainer(
    accelerator='cpu', 
    # devices=-1, # use all GPU's (-1)
    logger=wandb_logger,    # W&B integration
    max_epochs=3            # number of epochs
    )

trainer.fit(model, data)

trainer.test(model, datamodule=data)

wandb.finish()

wandb: Currently logged in as: christopher-marais. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name     | Type               | Params
------------------------------------------------
0 | layer_1  | Conv2d             | 30    
1 | layer_2  | Conv2d             | 168   
2 | layer_3  | Conv2d             | 660   
3 | pool     | MaxPool2d          | 0     
4 | layer_5  | Linear             | 30.0 M
5 | layer_6  | Linear             | 100 K 
6 | layer_7  | Linear             | 5.0 K 
7 | layer_8  | Linear             | 510   
8 | layer_9  | Linear             | 55    
9 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
30.1 M    Trainable params
0        

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


torch.Size([10, 30000])


IndexError: Target 6 is out of bounds.

# Parameter tuning sweep

In [ ]:
sweep_config = {
    "project": "mnist_bc",
    "method": "bayes",   # Random search
    "metric": {           # We want to maximize val_acc
        "name": "val_acc",
        "goal": "maximize"
    },
    "run_cap": 5, #terminates the sweep after a number of runs
    "early_terminate": { # only terminates a run early not the sweep (reduces computation time)
        "type": "hyperband",
        "min_iter": 3
    },
    "parameters": {
        # "n_layer_1": {
        #     # Choose from pre-defined values
        #     "values": [32, 64, 128, 256, 512]
        # },
        # "n_layer_2": {
        #     # Choose from pre-defined values
        #     "values": [32, 64, 128, 256, 512, 1024]
        # },
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -4.61    # exp(-4.61) = 1e-2
        }
    }
}

In [ ]:
def sweep_iteration():
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger()

    # setup data
    # data = MNISTDataModule()
    data = YogaDataModule()

    # setup model - note how we refer to sweep parameters with wandb.config
    # model = LitMNIST(
    #     n_layer_1=wandb.config.n_layer_1,
    #     n_layer_2=wandb.config.n_layer_2,
    #     lr=wandb.config.lr
    # )
    model = YogaModel(lr=wandb.config.lr, n_classes=num_of_classes)

    # setup Trainer
    trainer = Trainer(
        logger=wandb_logger,    # W&B integration
        gpus=-1,                # use all GPU's
        max_epochs=3            # number of epochs
        )

    # train
    trainer.fit(model, data)

In [ ]:
sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, function=sweep_iteration)